### With this interface you can ask a question and get an answer from the GPT, Claude and Gemini

In [49]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import time

In [2]:
import gradio as gr # oh yeah!

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAJ


In [4]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [6]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [7]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [8]:
def stream_gemini(prompt):
    gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
    )
    result = ""
    for response in gemini.generate_content(prompt, stream=True):
        result += response.text or ""
        yield result

In [92]:
def stream_models(prompt):
    response_gpt = ""
    response_claude =  ""
    response_gemini =  ""
    for gpt in stream_gpt(prompt):
        response_gpt = gpt
        yield response_gpt, response_claude, response_gemini
    for claude in stream_claude(prompt):
        response_claude = claude
        yield response_gpt, response_claude, response_gemini
    for gemini in stream_gemini(prompt):
        response_gemini = gemini
        yield response_gpt, response_claude, response_gemini

In [113]:
# Gradio interface
with gr.Blocks() as view:
    user_input = gr.Textbox(label="What models can help with?", placeholder="Type your question here")
    ask_button = gr.Button("Ask")
    with gr.Row():
        with gr.Column():
            gr.HTML(value="<b>GPT response:</b>") 
            gcp_stream = gr.Markdown()
        with gr.Column():
            gr.HTML(value="<b>Claude response:</b>") 
            claude_stream = gr.Markdown()
        with gr.Column():
            gr.HTML(value="<b>Gemine response:</b>") 
            gemini_stream = gr.Markdown()

    ask_button.click(
        fn=stream_models,  # Function that yields multiple outputs
        inputs=user_input,
        outputs=[gcp_stream, claude_stream, gemini_stream]  # Connect to multiple outputs
    )

view.launch()

* Running on local URL:  http://127.0.0.1:7919

To create a public link, set `share=True` in `launch()`.
